In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Read the csv file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ Data_pol2/2019_1000_merge_data.csv')

data['date_time'] = data['CalendarDateKey'].astype(str) + data['interval_start']
print(data.tail()) #7 columns, including the Date. 


         Unnamed: 0  Unnamed: 0.1   index  CalendarDateKey interval_start  \
1994387     1994387       1994387  3042.0         20190807       09:00:00   
1994388     1994388       1994388  3042.0         20190825       15:00:00   
1994389     1994389       1994389  3042.0         20191025       08:00:00   
1994390     1994390       1994390  3042.0         20191115       08:00:00   
1994391     1994391       1994391  3042.0         20191129       08:00:00   

         n_validations         date_time  
1994387              8  2019080709:00:00  
1994388             10  2019082515:00:00  
1994389              5  2019102508:00:00  
1994390              5  2019111508:00:00  
1994391              5  2019112908:00:00  


In [ ]:
import datetime
#process the matrix
df_for_training=data.pivot(index='date_time',
                  columns='index',
                  values='n_validations'
                  )
df_for_training.index = pd.to_datetime(df_for_training.index, format='%Y%m%d%H:%M:%S')


In [ ]:
df_for_training.index = pd.to_datetime(df_for_training.index, format='%Y%m%d%H:%M:%S')

In [ ]:
print('there are %d unrecorded Nans' % df_for_training.isna().sum().sum())
df_for_training=df_for_training.fillna(0)
df_for_training = df_for_training.astype(float)
df_for_training.shape
print('there are %d unrecorded Nans' % df_for_training.isna().sum().sum())

there are 4094243 unrecorded Nans
there are 0 unrecorded Nans


In [ ]:
df_for_training.head()  # row-time and column-grids

index,0.0,1.0,2.0,3.0,4.0,10.0,12.0,13.0,19.0,20.0,...,1329.0,1334.0,1335.0,1342.0,1347.0,1348.0,1350.0,1361.0,1362.0,1372.0
date_time,,,,,,,,,,,,,,,,,,,,,
2019-07-01 05:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-07-01 05:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
2019-07-01 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,13.0,6.0
2019-07-01 06:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0,0.0
2019-07-01 07:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.0,8.0,0.0,0.0,0.0


In [ ]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df_training, window_size=5,n_forcast = 1):

  X = []
  y = []
  for i in range(window_size, len(df_training) - n_forcast +1):
      X.append(df_training[i - window_size:i, 0:df_for_training.shape[1]])
      y.append(df_training[i + n_forcast - 1:i + n_forcast, :])
  return np.array(X), np.array(y)

In [ ]:
WINDOW_SIZE = 18
X1, y1 = df_to_X_y(df_for_training.to_numpy(), WINDOW_SIZE,n_forcast=1)
X1.shape, y1.shape

((5337, 18, 1137), (5337, 1, 1137))

In [ ]:
# measure the mse
def measure_the_error(actual,pred,period=0):

    mean_square=np.square(np.subtract(actual,pred)).mean()
    # mean_absolute_percentage=np.mean(np.abs((actual - pred)/actual))*100
    return mean_square

## stateful lstm

In [ ]:
# 70 training 15 validation 15 testing
X_train1, y_train1 = X1[:4287], y1[:4287]
# X_val1, y_val1 = X1[4270:4800], y1[4270:4800]
X_test1, y_test1 = X1[4287:], y1[4287:]
X_train1.shape, y_train1.shape, X_test1.shape, y_test1.shape

((4287, 18, 1137), (4287, 1, 1137), (1050, 18, 1137), (1050, 1, 1137))

In [ ]:
# model 2 stateful model
batch_size = 1 

model2 = Sequential()
model2.add(InputLayer(batch_input_shape = (batch_size,18,y_train1.shape[2])))
model2.add(LSTM(units=80,activation='relu',return_sequences=False,stateful = True))
model2.add(Dropout(0.2))
model2.add(Dense(1137))


model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (1, 80)                   389760    
                                                                 
 dropout_3 (Dropout)         (1, 80)                   0         
                                                                 
 dense_3 (Dense)             (1, 1137)                 92097     
                                                                 
Total params: 481,857
Trainable params: 481,857
Non-trainable params: 0
_________________________________________________________________


In [ ]:


model2.compile(optimizer = 'adam', loss='mean_squared_error')

for i in range(10):
    model2.fit(X_train1, y_train1, epochs=1, batch_size=batch_size,  shuffle=False)
    model2.reset_states()

4287/4287 [==============================] - 94s 22ms/step - loss: 8625.5352


In [ ]:
history = []
batch_size=1
for i in range(X_test1.shape[0]//batch_size):
  pre = model2.predict(X_test1[i,:,:].reshape(batch_size,18,1137),batch_size = batch_size,verbose = 0)
  history.append(pre[0])
pred = np.asarray(history)

In [ ]:
def measure_the_error(actual,pred):
    rmse_list=[]
    mae_list=[]
    num=int(len(pred)/35)
    root_mean_square=np.sqrt(np.square(np.subtract(actual,pred)).mean(axis=1))
    mean_absolute_error=abs(np.subtract(actual,pred)).mean(axis=1)
    for i in range(num):

        cur=35*i
        rmse_list.append(root_mean_square[cur:cur+35].mean())
        mae_list.append(mean_absolute_error[cur:cur+35].mean())
    # mean_absolute_percentage=np.mean(np.abs((actual - pred)/actual))*100
    # rmse = math.sqrt(mean_square)
    return rmse_list,mae_list


rmse,mae_list = measure_the_error(pred,y_test1.reshape(1050,1137))


In [ ]:
rmse

[41.10887578280718,
 40.468267969254576,
 29.23438085178533,
 51.855207258931934,
 53.52965732706335,
 53.56912556759614,
 53.914424379720096,
 53.918171689968275,
 34.822175621479836,
 28.88489054172339,
 53.73624766046661,
 53.26520476344116,
 53.89690844750153,
 55.627022979655784,
 51.88024715125388,
 34.96106811104971,
 29.21365601151757,
 52.417991562286765,
 54.892965546399054,
 52.48606777834332,
 54.93399313157754,
 54.857221330924126,
 35.572771263582986,
 30.51484928354741,
 51.8933841108132,
 53.65791578916836,
 54.26924423016843,
 56.4999820943571,
 63.99499466281991,
 44.23330925431815]